<a href="https://colab.research.google.com/github/digdemorhan/Model_Building_With_Functional_API/blob/main/ModelBuildingwithFunctionalAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VERİ SETİNİN YÜKLENMESİ




In [24]:
#Veri setini yükleme
from sklearn.datasets import fetch_california_housing

#Veri setinin değişkene atanması
housing = fetch_california_housing()

#Eğitim ve test verilerinin oluşturulması
from sklearn.model_selection import train_test_split

#Veri setinin parçalanması
X_train, x_test, y_train, y_test = train_test_split(housing.data, housing.target, random_state=42)


# MODELİN KURULMASI

In [25]:
#Modelin kurulması
import tensorflow as tf
tf.random.set_seed(42)

#Katmanların inşa edilmesi
normalization_layer = tf.keras.layers.Normalization() #Bu katman kullanıldığı için Flatten'a gerek yok
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1) #1 sayısal değer tahmin edileceği için 1 yazıldı

 ***Normalization Katmanı: Bu katman, Scikit-learn'deki StandardScaler'ın yaptığı işi yapar. Sayısal verilerin
normalleştirilmesine yardımcı olur. Bu katman için "adapt metodu" kullanılır. Bu metot, verilerin ortalaması ve varyansını hesaplar. (fit metodundan önce çağrılacaktır.)***
***Concat Katmanı: Katmanları birleştirmek için kullanılacaktır.***

# SİNİR AĞININ İNŞA EDİLMESİ

In [26]:
#Sinir ağının inşa edilmesi
input_ = tf.keras.layers.Input(shape=X_train.shape[1:]) #öznitelik sayısı
normalized = normalization_layer(input_) #veriler bu katmandan geçtikten sonra gizli katmana gidecek
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

#Modelin oluşturulması
model = tf.keras.Model(inputs=[input_], outputs=[output])

#Modelin özetinin görülmesi
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 8)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_8           │ (None, 8)              │             17 │ input_layer_7[0][0]    │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_23 (Dense)          │ (None, 30)             │            270 │ normalization_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_24 (Dense)          │ (None, 30)             │            930 │ dense_23[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_7             │ (None, 38)             │              0 │ normalization_8[0][0], │
│ (Concatenate)             │                        │                │ dense_24[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_25 (Dense)          │ (None, 1)              │             39 │ concatenate_7[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,256 (4.91 KB)

 Trainable params: 1,239 (4.84 KB)

 Non-trainable params: 17 (72.00 B)

# MODELİN COMPILE EDİLMESİ

In [27]:
#Optimizer'ın belirlenmesi
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
model.compile(
    loss = "mse",
    optimizer = optimizer,
    metrics = ["RootMeanSquaredError"]
)

# MODELİN EĞİTİLMESİ

In [28]:
#Eğitim verilerinin normalleştirilmesi
normalization_layer.adapt(X_train)
history = model.fit(
    X_train, y_train,
    validation_split = 0.2,
    epochs = 20
)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.3969 - loss: 2.0676 - val_RootMeanSquaredError: 0.7505 - val_loss: 0.5633
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6983 - loss: 0.4878 - val_RootMeanSquaredError: 0.6680 - val_loss: 0.4462
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6328 - loss: 0.4005 - val_RootMeanSquaredError: 0.6494 - val_loss: 0.4217
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.6104 - loss: 0.3727 - val_RootMeanSquaredError: 0.6340 - val_loss: 0.4020
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.5967 - loss: 0.3562 - val_RootMeanSquaredError: 0.6219 - val_loss: 0.3868
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - RootMeanSquaredError: 0.5859 - loss: 0.3435 - val_RootMeanSquaredError: 0.6123 - val_loss: 0.3749
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.5770 - los

# MODELİN DEĞERLENDİRİLMESİ

In [29]:
mse_test = model.evaluate(x_test, y_test)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - RootMeanSquaredError: 0.5532 - loss: 0.3062


# DEĞER TAHMİNİ

In [30]:
#Dataframe yapısına çevrilmesi için Pandas'ın import edilmesi
import pandas as pd

#Test verilerinin tahmini
X_new = x_test[:3]

#Dataframe'e çevrilmesi
pd.DataFrame(X_new)

,0,1,2,3,4,5,6,7
0,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01
1,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46
2,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44


In [31]:
#Predict metoduyla verilerin modele göre tahmin edilmesi
y_pred = model.predict(X_new)
pd.DataFrame(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step


,0
0,0.594415
1,1.095122
2,4.366732


In [32]:
#Tahmin edilen verilerin gerçek değerlerine bakılması
pd.DataFrame(y_test[:3])

,0
0,0.47700
1,0.45800
2,5.00001


# FARKLI INPUT'LAR KULLANILARAK MODELİN İNŞA EDİLMESİ

***Inputların farklı yollardan alındığı bir senaryo inşa edilecektir.***

In [33]:
#Girdi katmanlarının oluşturulması
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])

#İki farklı girdi için normalization'dan örnek alınması
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

#Girdi verileri için normalization katmanlarının oluşturulması
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

#Gizli katmanların oluşturulması
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

#Gizli ve normalization katmanlarının birleştirilmesi
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Çıktı katmanının oluşturulması
output = tf.keras.layers.Dense(1)(concat)

#Modelin oluşturulması
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

#Compile metodu ile modelin derlenmesi
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(
    loss = "mse",
    optimizer = optimizer,
    metrics = ["RootMeanSquaredError"]
)

#Girdi değişkenlerinin belirlenmesi (ilk 5 ve 2'den sonraki)
#Eğitim veri setlerinin oluşturulması
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_test_wide, X_test_deep = x_test[:, :5], x_test[:, 2:]

#Normalization'daki ort ve ss için adapt metodunun çağrılması
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

#Modelin eğitilmesi
history = model.fit(
    (X_train_wide, X_train_deep),
    y_train, epochs = 20,
    validation_split = 0.2
)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 1.4835 - loss: 2.3036 - val_RootMeanSquaredError: 0.7721 - val_loss: 0.5961
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.7146 - loss: 0.5109 - val_RootMeanSquaredError: 0.6786 - val_loss: 0.4605
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - RootMeanSquaredError: 0.6455 - loss: 0.4168 - val_RootMeanSquaredError: 0.6571 - val_loss: 0.4318
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - RootMeanSquaredError: 0.6258 - loss: 0.3917 - val_RootMeanSquaredError: 0.6442 - val_loss: 0.4150
Epoch 5/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - RootMeanSquaredError: 0.6143 - loss: 0.3775 - val_RootMeanSquaredError: 0.6342 - val_loss: 0.4022
Epoch 6/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - RootMeanSquaredError: 0.6053 - loss: 0.3664 - val_RootMeanSquaredError: 0.6256 - val_loss: 0.3914
Epoch 7/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.5973 - los

In [34]:
#Modelin değerlendirilmesi
mse_test = model.evaluate(
    (X_test_wide, X_test_deep),
    y_test
)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - RootMeanSquaredError: 0.5773 - loss: 0.3334


In [35]:
#Test veri setinden veri alarak tahmin yapılması
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]
y_pred = model.predict((X_new_wide, X_new_deep))
pd.DataFrame(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step


,0
0,0.517039
1,0.909208
2,3.872485


# ÇOKLU ÇIKTILAR İÇİN FUNCTIONAL API

In [56]:
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

#Gizli katmanların oluşturulması
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

#Concat katmanının oluşturulması
concat = tf.keras.layers.concatenate([norm_wide, hidden2])

#Çıktı katmanının oluşturulması
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)

#Modelin oluşturulması
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

#Modelin compile edilmesi
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(
    loss = ("mse","mse"),
    loss_weights = (0.9, 0.1),
    optimizer = optimizer,
    metrics = ["RootMeanSquaredError", "RootMeanSquaredError"]
)

#Girdi verilerinin normalization işlemi için adapt metodunun kullanılması
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

#Modelin eğitilmesi
history = model.fit(
    (X_train_wide, X_train_deep),
    (y_train, y_train),
    epochs = 20,
    validation_split = 0.2
)

Epoch 1/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - dense_71_RootMeanSquaredError: 1.3429 - dense_72_RootMeanSquaredError: 1.9282 - loss: 2.0731 - val_dense_71_RootMeanSquaredError: 0.7538 - val_dense_72_RootMeanSquaredError: 1.0405 - val_loss: 0.6196
Epoch 2/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - dense_71_RootMeanSquaredError: 0.7065 - dense_72_RootMeanSquaredError: 0.9575 - loss: 0.5412 - val_dense_71_RootMeanSquaredError: 0.6819 - val_dense_72_RootMeanSquaredError: 0.8550 - val_loss: 0.4916
Epoch 3/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - dense_71_RootMeanSquaredError: 0.6493 - dense_72_RootMeanSquaredError: 0.8151 - loss: 0.4459 - val_dense_71_RootMeanSquaredError: 0.6672 - val_dense_72_RootMeanSquaredError: 0.8002 - val_loss: 0.4647
Epoch 4/20
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - dense_71_RootMeanSquaredError: 0.6315 - dense_72_RootMeanSquaredError: 0.7675 - loss: 0.4179 - val_dense_71_RootMeanSquaredError: 0.6539 - val_dense_72_RootMeanSquaredError: 0.7725 - v

In [63]:
#Modelin değerlendirilmesi
eval_results = model.evaluate(
    (X_test_wide, X_test_deep),
    (y_test, y_test)
)

#Modelin değerlendirme sonuçlarının bir değişkene atanarak görüntülenmesi
weighted_sum_of_losses, main_loss, aux_loss = eval_results
print(eval_results)

#Predict metoduyla tahminlerin bulunması
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))
print("y_pred_main: ", y_pred_main)
print("y_pred_aux: ", y_pred_aux)

162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_71_RootMeanSquaredError: 0.5747 - dense_72_RootMeanSquaredError: 0.6298 - loss: 0.3371
[0.33389541506767273, 0.5724886655807495, 0.6239109039306641]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
y_pred_main:  [[0.5446641]
 [1.2990813]
 [3.959517 ]]
y_pred_aux:  [[0.68879753]
 [1.1646816 ]
 [3.7747288 ]]
